In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
from scipy import stats

# Load run 224 and 225 data

In [2]:
def load_data(run_id):
    file_name = "/reg/d/psdm/xpp/xppx40318/hdf5/smalldata/xppx40318_Run{}.h5".format(run_id)

    with h5.File(file_name, 'r') as h5file:
        #print(list(h5file['ipm2'].keys()))
        imp_value = np.array(h5file['ipm2/sum'])

        #print(list(h5file['ai'].keys()))
        analog_in = np.array(h5file["ai/ch01"])


    mask = np.invert(np.isnan(analog_in))
    mask[analog_in>=1.75] = False

    ipm2 = imp_value[mask]
    intensity_meter = analog_in[mask]
    
    return ipm2, intensity_meter

In [3]:
ipm2_ini, intensity_meter_ini = load_data(225)   # For ipm 2
ipm2_g2, intensity_meter_g2 = load_data(226)   # For g2
ipm2_sd, intensity_meter_sd = load_data(223)   # For after SD

/cds/sw/ds/ana/conda1/inst/envs/ana-4.0.11/lib/python2.7/site-packages/ipykernel/__main__.py:13: RuntimeWarning: invalid value encountered in greater_equal


# Get the norminal energy efficiency measured with the intensity meter

In [4]:
curve_ini = stats.linregress(y=intensity_meter_ini, x=ipm2_ini)
curve_g2 = stats.linregress(y=intensity_meter_g2 / 10., x=ipm2_g2 )
curve_sd = stats.linregress(y=intensity_meter_sd / 10., x=ipm2_sd )

In [6]:
def get_efficiency_and_uncertainty(direct_beam, diffraction):
    
    # Get efficiency
    _efficiency = diffraction.slope / direct_beam.slope
    #print("The energy efficiency is {:.2f}%".format(_efficiency * 100))
    
    # Get the relative efficiency of the efficiency
    relative_std = (diffraction.stderr / diffraction.slope) ** 2
    relative_std += (direct_beam.stderr / direct_beam.slope) ** 2
    relative_std = np.sqrt(relative_std)
    
    # Get the uncertainty of the efficiency
    _eff_std = _efficiency * relative_std
    
    #print("The standard deviation of the efficiency is {:.2f}%".format(100 * _eff_std))
    return _efficiency, _eff_std

In [9]:
eff_g2, eff_g2_std = get_efficiency_and_uncertainty(curve_ini, curve_g2)
eff_sd, eff_sd_std = get_efficiency_and_uncertainty(curve_ini, curve_sd)

print("The energy efficiency after g2 is {:.2f}%. The uncertainty is {:.2f}%".format(100 * eff_g2, 100 *eff_g2_std))
print("The energy efficiency after Sd is {:.2f}%. The uncertainty is {:.2f}%".format(100 * eff_sd, 100 *eff_sd_std))

The energy efficiency after g2 is 5.99%. The uncertainty is 0.02%
The energy efficiency after Sd is 11.13%. The uncertainty is 0.03%


## Because there are systematic error in the intensity meter. Therefore, manually adjust the uncertainty to be 10% of the absolute value

In [10]:
eff_g2_std += 0.1 * eff_g2
eff_sd_std += 0.1 * eff_sd

print("After manual adjustment, the result become:")
print("The energy efficiency after g2 is {:.2f}%. The uncertainty is {:.2f}%".format(100 * eff_g2, 100 *eff_g2_std))
print("The energy efficiency after Sd is {:.2f}%. The uncertainty is {:.2f}%".format(100 * eff_sd, 100 *eff_sd_std))

After manual adjustment, the result become:
The energy efficiency after g2 is 5.99%. The uncertainty is 0.62%
The energy efficiency after Sd is 11.13%. The uncertainty is 1.15%


# Calculate the energy efficiency of the second gratings

In [20]:
diamond_coef = 7.3453 / 1e4

# Calculate the transmission
g2_thickness = 100 / np.cos(np.deg2rad(72))
g2_transmission = np.exp( - g2_thickness * diamond_coef)

# Calculate the uncertainty of the transmission
# Get the upper limit
g2_thickness_ul = 100 / np.cos(np.deg2rad(70))  
g2_transmission_ul = np.exp( - g2_thickness_ul * diamond_coef)
# Get the lower limit
g2_thickness_ll = 100 / np.cos(np.deg2rad(74))  
g2_transmission_ll = np.exp( - g2_thickness_ll * diamond_coef)

g2_trans_std = (g2_transmission_ul - g2_transmission_ll) / 2.

print("The g2 transmission efficiency is {:.2f} % with uncertainty {:.2f}%".format(g2_transmission * 100 ,
                                                                                        g2_trans_std * 100))

The g2 transmission efficiency is 78.84 % with uncertainty 2.03%


## Because after g2, one has measured the total energy rather than energy in the +-1 order of diffraction. Therefore, this needs correction

In [21]:
# Assume the +-1 order of diffraction has energy efficiency of 20% with uncertainty of 2 %.
eff_g2_diff = 0.2
eff_g2_diff_std = 0.02

# Get the energy efficiency of the total device
eff_g2_corrected = eff_g2 / g2_transmission * eff_g2_diff

# Get the uncertainty of this estimation
eff_g2_corrected_std = ((eff_g2_std / eff_g2) ** 2 + 
                        (g2_trans_std / g2_transmission) ** 2 + 
                        (eff_g2_diff_std / eff_g2_diff)**2)
eff_g2_corrected_std = np.sqrt(eff_g2_corrected_std)
eff_g2_corrected_std *= eff_g2_corrected

print("After currection, the result become:")
print("The energy efficiency after g2 is {:.2f}%. The uncertainty is {:.2f}%".format(100 * eff_g2_corrected, 
                                                                                     100 * eff_g2_corrected_std))

After currection, the result become:
The energy efficiency after g2 is 1.52%. The uncertainty is 0.22%


In [33]:
# calculate the uncertainty of double grating

In [35]:
uncertainty = ((eff_g2_diff_std / eff_g2_diff) ** 2 + (eff_g2_diff_std / eff_g2_diff) ** 2)
uncertainty = eff_g2_diff * 2 * eff_g2_diff * uncertainty
print(uncertainty)

0.0016


# Calculate the energy efficiency of the SD table

In [23]:
# The absorption coefficient of different material
kap_coef = 1. / 2257.9 # convert to um
air_coef = 0.618 / 1e6 # convert to um
diamond_coef = 7.3453 / 1e4

In [28]:
# List the length of materials before the SD table
kap_len = 60
air_len = 0.16 * 1e6 # Convert to um

# Assume that the first grating has the same behavior as the second grating with considering the energy efficiency

# Calculat the energy efficiency
sd_upstream_energy = g2_transmission * np.exp(-air_len * air_coef) * np.exp(-kap_len * kap_coef)

# Calculate the uncertainties
air_eff = np.exp(-air_len * air_coef)
air_eff_std = (np.exp(-(air_len - 0.02 * 1e6) * air_coef) - np.exp(-(air_len + 0.02 * 1e6) * air_coef)) / 2.

kap_eff = np.exp(-kap_len * kap_coef)
kap_eff_std = (np.exp(-(kap_len - 3) * kap_coef) - np.exp(-(kap_len +3) * kap_coef)) / 2.

# Calculate the total uncertainty
sd_upstream_energy_std = ((g2_trans_std / g2_transmission) ** 2 +
                          (air_eff_std / air_eff) ** 2 +
                          (kap_eff_std / kap_eff) ** 2)
sd_upstream_energy_std = np.sqrt(sd_upstream_energy_std)
sd_upstream_energy_std *= sd_upstream_energy

print("After estimation")
print("The energy efficiency before sd is {:.2f}%. The uncertainty is {:.2f}%".format(100 * sd_upstream_energy, 
                                                                                     100 * sd_upstream_energy_std))

After estimation
The energy efficiency before sd is 69.55%. The uncertainty is 1.99%


In [29]:
# Calculate the energy efficiency of the sd table
sd_table_eff = eff_sd / sd_upstream_energy / (eff_g2_diff * 2) * g2_transmission

sd_table_eff_std = ((eff_sd_std / eff_sd) ** 2 +
                    (sd_upstream_energy_std / sd_upstream_energy) ** 2 +
                    (eff_g2_diff_std / eff_g2_diff) ** 2 +
                    (g2_trans_std / g2_transmission) ** 2)

sd_table_eff_std = np.sqrt(sd_table_eff_std)
sd_table_eff_std *= sd_table_eff
print("After estimation")
print("The energy efficiency of sd table is {:.2f}%. The uncertainty is {:.2f}%".format(100 * sd_table_eff, 
                                                                                     100 * sd_table_eff_std))

After estimation
The energy efficiency of sd table is 31.53%. The uncertainty is 4.69%


# Calculate the energy efficiency of the channel-cut crystals directly

In [32]:
##################################################################################
#       Kapton coef
##################################################################################
vcc_kapton_length = 30 * 2 + 125 + 3 * 50.
cc_kapton_length = 30 * 2 + 100.

kap_len =   ( vcc_kapton_length + cc_kapton_length ) / 2 / 1e6
kap_len_std = (vcc_kapton_length - cc_kapton_length + 30 ) / 2 / 1e6

kap_eff = np.exp(-kap_coef * kap_len)
kap_eff_std = np.exp(-kap_coef * (kap_len - kap_len_std)) - np.exp(-kap_coef * (kap_len + kap_len_std))

##################################################################################
#        Air coef
##################################################################################
air_len = 0.1 + 1.2 * 0.1 # Conver the air path in the helium to pure air
air_len *= 1e6

air_eff = np.exp(-air_len * air_coef)

air_eff_std = (np.exp(-(air_len - 0.05 * 1e6) * air_coef) - np.exp(-(air_len + 0.05 * 1e6) * air_coef)) / 2.

##################################################################################
#        Diamend coef
##################################################################################
diamond_thickness =  60

diamond_eff = np.exp(- diamond_coef * diamond_thickness)
diamond_eff_std = (np.exp(- diamond_coef * (diamond_thickness - 2)) - np.exp(- diamond_coef * (diamond_thickness + 2))) / 2

###########################################################
#      Calculate the Channel-cut efficiency
###########################################################
cc_eff = sd_table_eff / diamond_eff / air_eff / kap_eff

cc_eff_std = ((sd_table_eff_std / sd_table_eff) ** 2 +
              (diamond_eff_std / diamond_eff) ** 2 +
              (air_eff_std / air_eff) ** 2 + 
              (kap_eff_std / kap_eff) ** 2)
cc_eff_std = np.sqrt(cc_eff_std)
cc_eff_std *= cc_eff

print("After estimation")
print("The energy efficiency of CC is {:.2f}%. The uncertainty is {:.2f}%".format(100 * cc_eff, 
                                                                                     100 * cc_eff_std))

After estimation
The energy efficiency of CC is 37.75%. The uncertainty is 5.73%


# Calculate the loss of other components

In [36]:
loss = eff_g2_corrected / cc_eff / (eff_g2_diff * 2 * eff_g2_diff)

loss_std = ((eff_g2_corrected_std / eff_g2_corrected) ** 2 +
            (cc_eff_std / cc_eff) ** 2+
            4 * (eff_g2_diff_std / eff_g2_diff))
loss_std = loss * np.sqrt(loss_std)

print(loss, loss_std)

(0.503231566213772, 0.3354591062229563)
